# User Code 

This notebook takes user video , processes the key frames to compare them to the trainer frames stored in csv 

# Import Libraries

In [1]:
import cv2
import os
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import csv

pose_model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = pose_model.signatures['serving_default']

key_frames_angles = []
key_frames_points = []
mapping_frames = []


In [2]:
excersise_name = "PushUp"
min_l1_norm = 45 

In [3]:
csv_file = os.path.join(excersise_name,"key_frame_angles.csv")

with open(csv_file, "r") as f:
    reader = csv.DictReader(f)
    data = [dict(row) for row in reader]

csv_file = os.path.join(excersise_name,"key_frame_points.csv")

with open(csv_file, "r") as f:
    reader = csv.DictReader(f)
    data2 = [dict(row) for row in reader]


In [4]:
data2

[{'nose': '[0.23615095 0.5675388  0.46265757]',
  'left_shoulder': '[0.27228904 0.52820957 0.64648396]',
  'right_shoulder': '[0.27421942 0.51909065 0.6864313 ]',
  'left_elbow': '[0.3442916  0.50258553 0.29173014]',
  'right_elbow': '[0.34716612 0.49362516 0.5878885 ]',
  'left_wrist': '[0.41239384 0.5306399  0.2780091 ]',
  'right_wrist': '[0.41443628 0.5181177  0.63111657]',
  'left_hip': '[0.46104944 0.5120772  0.64338255]',
  'right_hip': '[0.46171716 0.5057313  0.7075274 ]',
  'left_knee': '[0.5742486 0.5067785 0.6014585]',
  'right_knee': '[0.56939036 0.5093329  0.5548667 ]',
  'left_ankle': '[0.67054033 0.49951023 0.57965326]',
  'right_ankle': '[0.68115395 0.49426022 0.7055171 ]',
  'neck': '[0.27325422 0.5236501  0.66645765]'},
 {'nose': '[0.23587504 0.5629189  0.36062104]',
  'left_shoulder': '[0.2752046  0.52440625 0.7217681 ]',
  'right_shoulder': '[0.27527547 0.5157276  0.5780156 ]',
  'left_elbow': '[0.34314305 0.49532366 0.3573343 ]',
  'right_elbow': '[0.34598526 0.488

In [5]:
key_frames_angles = [[float(d[key]) for key in d] for d in data]
key_frames_points = [[d[key] for key in d] for d in data2]

In [6]:
key_frames_points = [[[float(co) for co in part.strip("[]").split() ] for part in frame] for frame in key_frames_points]

In [7]:
### Returns key points by taking model output on a image
def get_keypoints(outputs):
    points = outputs['output_0'].numpy()[0, 0, :, :]
    nose = points[0]
    left_shoulder = points[5]
    right_shoulder = points[6]
    left_elbow = points[7]
    right_elbow = points[8]
    left_wrist = points[9]
    right_wrist = points[10]
    left_hip = points[11]
    right_hip = points[12]
    left_knee = points[13]
    right_knee = points[14]
    left_ankle = points[15]
    right_ankle = points[16]
    neck = (left_shoulder + right_shoulder) / 2

    body_parts = {
        'nose': nose,
        'left_shoulder': left_shoulder,
        'right_shoulder': right_shoulder,
        'left_elbow': left_elbow,
        'right_elbow': right_elbow,
        'left_wrist': left_wrist,
        'right_wrist': right_wrist,
        'left_hip': left_hip,
        'right_hip': right_hip,
        'left_knee': left_knee,
        'right_knee': right_knee,
        'left_ankle': left_ankle,
        'right_ankle': right_ankle,
        'neck': neck
    }
    return body_parts

In [8]:
def cosine_angle(v1, v2):
    numer = np.dot(v1, v2)
    denom = np.linalg.norm(v1) * np.linalg.norm(v2)
    if denom == 0:
        if np.array_equal(v1, v2):
            return 0.0
        else:
            return 180.0
    degrees = np.degrees(np.arccos(numer / denom))
    return degrees

def calculate_angle(point1,point2,point3):
    v1 = point1[:2] - point2[:2]
    v2 = point3[:2] - point2[:2]
    return cosine_angle(v1, v2)

In [9]:
## Get angles from key points. 
def get_angles(keypoints):
    n_n_rs = calculate_angle(keypoints['nose'], keypoints['neck'], keypoints['right_elbow'])
    n_n_ls = calculate_angle(keypoints['nose'], keypoints['neck'], keypoints['left_elbow'])
    n_rs_re = calculate_angle(keypoints['neck'], keypoints['right_shoulder'], keypoints['right_elbow'])
    n_ls_le = calculate_angle(keypoints['neck'], keypoints['left_shoulder'], keypoints['left_elbow'])
    rs_re_rw = calculate_angle(keypoints['right_shoulder'], keypoints['right_elbow'], keypoints['right_wrist'])
    ls_le_lw = calculate_angle(keypoints['left_shoulder'], keypoints['left_elbow'], keypoints['left_wrist'])
    n_rh_rk = calculate_angle(keypoints['neck'], keypoints['right_hip'], keypoints['right_knee'])
    n_lh_lk = calculate_angle(keypoints['neck'], keypoints['left_hip'], keypoints['left_knee'])
    rh_rk_ra = calculate_angle(keypoints['right_hip'], keypoints['right_knee'], keypoints['right_ankle'])
    lh_lk_la = calculate_angle(keypoints['left_hip'], keypoints['left_knee'], keypoints['left_ankle'])

    angles = {
        "Nose-Neck-Right Shoulder": n_n_rs,
        "Nose-Neck-Left Shoulder": n_n_ls,
        "Neck-Right Shoulder-Right Elbow": n_rs_re,
        "Neck-Left Shoulder-Left Elbow": n_ls_le,
        "Right Shoulder-Right Elbow-Right Wrist": rs_re_rw,
        "Left Shoulder-Left Elbow-Left Wrist": ls_le_lw,
        "Neck-Right Hip-Right Knee": n_rh_rk,
        "Neck-Left Hip-Left Knee": n_lh_lk,
        "Right Hip-Right Knee-Right Ankle": rh_rk_ra,
        "Left Hip-Left Knee-Left Ankle": lh_lk_la
    }

    return angles

In [10]:
def get_data(image):
    image_tensor = tf.image.resize_with_pad(image, 192, 192)
    image_tensor = tf.cast(image_tensor, dtype=tf.int32)
    image_tensor = tf.expand_dims(image_tensor, axis=0)
    
    # Run the model.
    outputs = movenet(image_tensor)
    keypoints = outputs['output_0'].numpy()[0, 0, :, :]
    
    return keypoints

In [11]:

def l1_norm(l1, l2):
    if len(l1) != len(l2):
        raise ValueError("Lists must be of the same length.")
    return np.sum(np.abs(np.array(l1) - np.array(l2)))

def get_angles_from_model(frame):
    # Convert the frame from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Convert the frame to a TensorFlow tensor
    image = tf.convert_to_tensor(frame_rgb, dtype=tf.int32)
    
    image_tensor = tf.image.resize_with_pad(image, 192, 192)
    image_tensor = tf.cast(image_tensor, dtype=tf.int32)
    image_tensor = tf.expand_dims(image_tensor, axis=0)

    # Run the model.
    outputs = movenet(image_tensor)
    keypoints = get_keypoints(outputs)
    angles = get_angles(keypoints)
    return angles,keypoints

  
def process_frame(frame, frame_id_from_trainer):
    angles1,keypoints = get_angles_from_model(frame)
    angles_only = [float(angles1[key]) for key in angles1]
    # closest_frame = None
    match  = -1
    if(frame_id_from_trainer == len(key_frames_angles)):
        return 0
    d = l1_norm(key_frames_angles[frame_id_from_trainer], angles_only)
    if d<min_l1_norm:
        match = frame_id_from_trainer 
    else:
        match = -2
    
    mapping_frames.append({'user':keypoints,'trainer':key_frames_points[match],'matched': True if match != -2 else False})
    ## Show Key points
    return match + 1
    

In [12]:
def main():
    video = cv2.VideoCapture("./data/Hantelcurl.gif")
    count = 0
    wait_count = 0
    frame_id = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break
        
        # Skip 4 out of 5 frames
        a = process_frame(frame, frame_id)

        if a != -1:
            frame_id = a
        else:
            wait_count += 1
        
        if wait_count > 15:
            frame_id += 1

        if (frame_id == 0):
            count += 1
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [13]:
X = np.array([d['trainer'] for d in mapping_frames])
Y = np.array([d['user'] for d in mapping_frames])
Y = np.array([[frame[key] for key in frame] for frame in Y])

In [21]:
print(X.shape)
print(Y.shape)

(40, 14, 3)
(40, 14, 3)


In [22]:
X[0][0]

array([0.23382221, 0.58503777, 0.41816887])

In [28]:
import numpy as np

def convert_to_2d(frame):
    sliced_matrix = frame[:, :, :2]  # This will have the shape (40, 14, 2)
    reshaped_matrix = sliced_matrix.reshape(-1, 2)  # Reshape to (40*14, 2)
    return reshaped_matrix
X1 = convert_to_2d(X)
Y1 = convert_to_2d(Y)

In [29]:
# We will show positions to user based on trainer video. For that affine transformation from trainer to user should be done
import numpy as np
from scipy.optimize import least_squares

def affine_transformation(X, Y):
    n, m = X.shape

    def residuals(params):
        A = params[:m*m].reshape(m, m)
        b = params[m*m:]
        return (Y - (X @ A + b)).flatten()

    initial_guess = np.zeros((m * m) + m)
    result = least_squares(residuals, initial_guess)
    optimized_params = result.x
    A_opt = optimized_params[:m*m].reshape(m, m)
    b_opt = optimized_params[m*m:]
    
    return A_opt, b_opt

A_opt, b_opt = affine_transformation(X1, Y1)

print("Optimized A:")
print(A_opt)
print("Optimized b:")
print(b_opt)

Optimized A:
[[ 1.39990574  0.05504695]
 [-0.8981222   0.13475636]]
Optimized b:
[0.37508129 0.42340222]


In [30]:
video2 = cv2.VideoCapture('wrong_pushup.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
fps = int(video2.get(cv2.CAP_PROP_FPS))
frame_width = int(video2.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video2.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_writer = cv2.VideoWriter('wrong_pushup.mp4', fourcc, fps, (frame_width, frame_height))

# for d in mapping_frames:
#     y, x, confidence = keypoint
#     if confidence > 0.3:
#         x = int(x * frame_width)
#         y = int(y * frame_height)
#         cv2.circle(frame1, (x, y), 5, (0, 0, 255), -1)

image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
image = tf.convert_to_tensor(image,dtype=tf.int32)
keypoints = get_data(image)



OpenCV: Couldn't read video stream from file "wrong_pushup.mp4"


NameError: name 'frame' is not defined

In [31]:
!pwd

/Users/Saketh/Desktop/GymBros/GymBro
